#  Orientation data - angle analysis 
Analysis of the angles in 2D, then possibly 3D.
Goal: find angular velocity to compare with event log. 

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
#sns.set(color_codes=True)

# loading data
dfOri = {}
fnameClean = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU_clean.h5',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC_clean.h5', ]

path = '../datasets/'

for i, x in enumerate(fnameClean):
    dfOri[i] = pd.read_hdf(path+fnameClean[i], 'Orientation')
    
dfV = {}
# extraction of dT
for i, x in enumerate(fnameClean):
    dfV[i] = dfOri[i].diff().dropna()
    dfOri[i]['dT'] = dfV[i].Time


In [2]:
# in use function fecition
def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

def angle_theta(V1, V2):
    V1 = np.array(V1)
    V2 = np.array(V2)
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return Theta

def decompose_vec(X, Y, Z):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([X, Y, Z])  
    Vz = np.array([X, Y, 0])  
    Vx = np.array([0, Y, Z])

    # find the angles between them and orientation vector 
    LenghtVz, alpha = angle_vec(Vo, Vz)
    LenghtVx, gamma = angle_vec(Vo, Vx)
    #Vy = np.array([X, 0, Z])
    #LenghtVy, beta  = angle_vec(Vo, Vy)
    
    return LenghtVx, 0, LenghtVz


In [3]:
# dataframe for angular velocity around y axis analysis 
dfV = {}

dfV = dfOri[2]
dfV['vec']   = dfV.apply(lambda row: decompose_vec(row.X, row.Y, row.Z) , axis=1)
dfV['vec1']  = dfV.vec.shift(1)

dfV['theta'] = dfV.apply(lambda row: angle_theta(row.vec, row.vec1) , axis=1).dropna()
#dfV['theta'] = dfV.apply(lambda row: angle_vec(row.vec, row.vec1) , axis=1).dropna()
#dfV['theta'] = dfV['theta'].apply(lambda x: x[1])
dfV['lx'], dfV['ly'], dfV['lz'] = \zip(*df['num'].map(powers))
dfV.dropna(inplace=True)

#dfV.drop({'X','Y','Z'}, axis=1, inplace=True)
#print(dfV.keys())

ValueError: Shape of passed values is (150798, 3), indices imply (150798, 8)

In [ ]:
dfV['U'] = dfV.apply(lambda row: np.absolute(row.theta) / row.dT, axis=1)
dfV.dropna(inplace=True)

In [ ]:
# U and theta distributions
#print(dfV.keys())
i=100

fig = plt.figure(figsize=(9,9))
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)

print(dfV.theta)
sns.distplot(dfV.theta[1], ax=ax1)
sns.distplot(dfV.U    , ax=ax2)
ax3.scatter (dfV.Time , dfV.U)

fig = plt.figure()
ax = fig.add_subplot(111)
c = ax.plot(dfV.Time[0:i], dfV.U[0:i])


In [ ]:
# applying method to all experiments
dfV = {}

for i, x in enumerate(fnameClean):
    dfV[i] = dfOri[i]
    dfV[i]['vec']   = dfV[i].apply(lambda row: decompose_vec(row.X, row.Y, row.Z) , axis=1)
    dfV[i]['vec1']  = dfV[i].vec.shift(1)
    dfV[i]['theta'] = dfV[i].apply(lambda row: angle_vec(row.vec, row.vec1) , axis=1).dropna()
    #dfV[i]['theta'] = dfV[i]['theta'].apply(lambda x: x[1])
    dfV[i]['U']     = dfV[i].apply(lambda row: np.absolute(row.theta[1]) / row.dT, axis=1)
    dfV[i].dropna(inplace=True)
    dfV[i].drop({'X','Y','Z'}, axis=1, inplace=True)
    #print(dfV[i].keys())
    

In [ ]:
# angular velocity distribution for all experiments
fig = plt.figure(figsize=(9,12))
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412)

for i, x in enumerate(fnameClean):
    sns.distplot(dfV[i].theta[1]  , ax=ax1)
    sns.distplot(dfV[i].U  , ax=ax2)